# morse alphabet 

In [ ]:
import time
import sys

In [ ]:
class MorseAlphabet():

    def __init__(self):
        self.letters = '''A  .- 	 B  -... 	 C  -.-. 	 D  -..
        E  . 	 F  ..-. 	 G  --. 	 H  ....
        I  .. 	 J  .--- 	 K  -.- 	 L  .-..
        M  -- 	 N  -. 	 O  --- 	 P  .--.
        Q  --.- 	 R  .-. 	 S  ... 	 T  -
        U  ..- 	 V  ...- 	 W  .-- 	 X  -..-
        Y  -.-- 	 Z  --..'''
        self.encoded_letters_map = self.__generate_alphabet_map()
        self.encoded_letters = list(self.encoded_letters_map.values())
       
    def get_encoded_letters(self):
        return self.encoded_letters
        
    def get_encoded_letters_map(self):
        return self.encoded_letters_map
        
    def encode(self, letter):
        return self.encoded_letters_map[letter]
        
    def __generate_alphabet_map(self):
        key = None
        alphabet_map = {}
        for value in self.letters.split():
            if not key is None:
                alphabet_map[key] = value
                key = None
            else:
                key = value  
        return alphabet_map     

In [ ]:
morse = MorseAlphabet()

assert morse.encode('D') == '-..'
assert '-..' in morse.get_encoded_letters()
assert morse.get_encoded_letters_map()['D'] == '-..'

assert morse.encode('I') == '..'
assert '.---' in morse.get_encoded_letters()
assert morse.get_encoded_letters_map()['I'] == '..'

assert morse.encode('J') == '.---'
assert '.---' in morse.get_encoded_letters()
assert morse.get_encoded_letters_map()['J'] == '.---'

In [ ]:
class WordEncoder():
    def __init__(self, sep='', alphabet=None):
        self.encoded_dict = {}
        self.encoded_partial_dict = {}
        self.sep = sep
        if alphabet:
            self.alphabet = alphabet 
        else:
            self.alphabet = MorseAlphabet()
        
    def get_alphabet(self):
        return self.alphabet
        
    def get_list(self):
        return list(self.encoded_dict.keys())
        
    def get_dict(self):
        return self.encoded_dict
        
    def get_partial_dict(self):
        return self.encoded_partial_dict
        
    def add_word_list(self, words):
        [self.add_word(w) for w in words]
        
    def add_word(self, word):
        alphabet_map = self.alphabet.get_encoded_letters_map()
        letters = []
        for letter in word: 
            letters.append(alphabet_map[letter])
        encoded_word = self.sep.join(letters)
        for l in range(len(letters)): 
            partial_word = self.sep.join(letters[0:l])
            if partial_word in self.encoded_partial_dict: 
                if encoded_word not in self.encoded_partial_dict[partial_word]:
                    self.encoded_partial_dict[partial_word].append(encoded_word)
            else: 
                self.encoded_partial_dict[partial_word] = [encoded_word]
        if encoded_word in self.encoded_dict: self.encoded_dict[encoded_word] += 1
        else: self.encoded_dict[encoded_word] = 1

In [ ]:
encoder = WordEncoder()

encoder.add_word('HTE')
encoded_dict = encoder.get_dict()
assert encoded_dict['....-.'] == 1
encoder.add_word('HTE')
encoded_dict = encoder.get_dict()
assert encoded_dict['....-.'] == 2
assert len(encoder.get_list()) == 1


encoder = WordEncoder(sep='|', alphabet=morse)
encoder.add_word('HTE')
encoded_dict = encoder.get_dict()
assert encoded_dict['....|-|.'] == 1

encoder = WordEncoder()
encoder.add_word_list(['HTE', 'EE'])
encoded_dict = encoder.get_dict()
assert encoded_dict['....-.'] == 1
assert encoded_dict['..'] == 1
assert len(encoder.get_list()) == 2

#### version 5 6 
- basic reducer with remaining
- replace list manipulations with node -> still bumpinto the reccursion  limitation
- replace reccursion with command pattern -> still too long 
- options object - keep track of solved positions

In [ ]:
class Context():
    def __init__(self, word_encoder=None): 
        if word_encoder:
            self.word_encoder = word_encoder            
        else:
            self.word_encoder = WordEncoder()
    
    def get_sentence(self):
        return self.sentence    
    
    def get_encoder(self):
        return self.word_encoder

In [ ]:
class Node():
    def __init__(self, segment, duplicates=1, parent=None):
        self.parent = parent
        self.segment = segment
        self.duplicates = duplicates
        self.length = len(segment)
        
    def __repr__(self):
        return f'<Node>{self.segment}'
        
    def get_segment(self):
        return self.segment
    
    def get_duplicates(self):
        return self.duplicates
        
    def get_sequence(self, sep='|'):
        sequence = [self.segment]
        parent = self.parent
        while parent:
            sequence.append(parent.segment)
            parent = parent.parent
        sequence.reverse()
        text = sep.join(sequence)
        return text
    
    def get_length_sequence(self):
        sequence = [self.length]
        parent = self.parent
        while parent:
            sequence.append(parent.length)
            parent = parent.parent
        sequence.reverse()    
        return sequence
    
    def get_sum_length_sequence(self):
        return sum(self.get_length_sequence())    
    
    def get_nb_options(self):
        nb_options = self.duplicates
        parent = self.parent
        while parent:
            nb_options *= parent.duplicates
            parent = parent.parent
        return nb_options
    
    def append(self, node):
         node.parent = self
        
    def get_sequence_start(self):
        node = self
        while node.parent:
            node = node.parent
        return node


In [ ]:
node_1 = Node('..')
print(node_1.get_sequence())
assert node_1.get_sequence() == '..'
assert node_1.get_sum_length_sequence() == 2

print("nodes")
node_2 = Node('-', parent=node_1)
node_3 = Node('.--', parent=node_2)
assert node_3.get_sequence() == '..|-|.--'
assert node_3.get_sum_length_sequence() == 6
assert node_2.get_sum_length_sequence() == 3

print("append")
node_4 = Node('.')
node_3.append(node_4)
assert node_4.get_sequence() == '..|-|.--|.'

print("start")
origin = node_4.get_sequence_start()
assert origin.get_sequence() == '..'

print("options")
node_5 = Node('--', parent=node_4, duplicates=2)
print(node_5.get_nb_options())
assert node_5.get_nb_options() == 2


In [ ]:
class Command():
    def __init__(self, level, pos, last_node, remaining):
        self.pos = pos
        self.level = level
        self.last_node = last_node
        self.remaining = remaining
        self.done = True if not remaining else False
    
    def get_level(self):
        return self.level     
    def get_pos(self):
        return self.pos  
    def get_last_node(self):
        return self.last_node     
    def get_remaining(self):
        return self.remaining    
    def get_done(self):
        return self.done 

In [ ]:
import operator
import functools

class Options():
    def __init__(self, size):
        self.size = size
        self.slots = [{} for i in range(size)]
            
    def append(self, pos, node) -> bool:
        added = False
        if node.get_segment() not in self.slots[pos]:
            # avoid counting the same segment twice - will prune
            self.slots[pos][node.get_segment()] = node.get_duplicates()
            added = True
        return added
        
    def count(self):
        # duplicates multiply
        # nb nodes in slot add up duplicates
        slots_count = [sum(slot.values()) for slot in self.slots]
        total = functools.reduce(operator.mul, [n for n in slots_count if n>0], 1)
        return total
        
    def pretty_print(self):
        text = ''
        [print(slot) for slot in self.slots]
        return text

In [ ]:
words = [ 'ABC', 'DEF', 'AB', 'CD', 'EF']
sentence = 'ABCDEF'
options = Options(len(sentence))
options.append(0, Node('ABC'))
options.append(3, Node('DEF'))
options.append(0, Node('AB'))
options.append(2, Node('CD'))
options.append(4, Node('EF'))
print(options.pretty_print())
assert options.slots[0] == {'ABC': 1, 'AB': 1}

assert options.count() == 2 # 'ABC' 'DEF', 'AB' 'CD' 'EF'

res = options.append(4, Node('EF'))
assert res is False
assert options.slots[4] == {'EF': 1}

options = Options(len(sentence))
options.append(0, Node('ABC', duplicates=2)) # lets predent ABC actually stands for 2 diffenent encoded words
options.append(3, Node('DEF'))
options.append(0, Node('AB'))
options.append(2, Node('CD'))
options.append(4, Node('EF'))

assert options.count() == 3 # 'ABC'*2 'DEF', 'AB' 'CD' 'EF'


In [ ]:
class SentenceDecoder():
    def __init__(self, context):
        self.context = context
        self.sentence = None
        self.valid_words_map = context.get_encoder().get_dict()
        self.options = None
        #self.partial_words_map  = context.get_encoder().get_partial_dict()
        #self.morse_letters = context.get_encoder().get_alphabet().get_encoded_letters()

    def screen_words(self, encoded_words_map):
        valid_encoded_words_maps = { k:v for k,v in encoded_words_map.items() if self.sentence.find(k)>=0}
        print(f'nb possible words {len(valid_encoded_words_maps.keys())}')
        return valid_encoded_words_maps
   
    def decode(self, sentence):
        # dumb cases
        if len(sentence) == 0: 
            print('empty sentence')
            return 0      
        
        self.sentence = sentence
        
        self.valid_words_map = self.screen_words(self.valid_words_map)
        if len(self.valid_words_map.keys()) == 0: 
            print('empty words')
            return 0
        #print(f'valid_words_map {self.valid_words_map}')

        self.options = Options(len(sentence))
        start_command = Command(0, 0, None, self.sentence)
        commands = [start_command]
        nb_options = 0
        while commands:
            commands, nb_options = self.decode_loop(commands, nb_options)
            
        return nb_options

    def decode_loop(self, commands, last_nb_options):
        nb_options = last_nb_options
        new_commands = []
        new_sequences = {}
        for command in commands:
            level = command.get_level()
            last_node = command.get_last_node()
            remaining = command.get_remaining()
            pos = command.get_pos()
            done = command.get_done()
            #if last_node:
            #   print(f'cmdi.{level} current:{last_node.get_sequence()} remaining:{remaining} done:{done}')
            
            if command.get_done(): 
                nb_options_for_sequence = last_node.get_nb_options()  
                print(f'done.{level} current:{last_node.get_sequence()} remaining:{remaining} nb:{nb_options_for_sequence}')
                nb_options += nb_options_for_sequence 
                continue
                 
            for word, duplicates in self.valid_words_map.items():
                if remaining.startswith(word):
                    new_last_node = Node(word, duplicates=duplicates)
                    if last_node: 
                        last_node.append(new_last_node)
                        #il faut trouver un moyen d'oliùoner les mots déjatrouvés comme ça
                        #ndex de szq de nodes
                        #new_sequences[new_last_node.get_sequence(sep='')] = new_last_node.get_nb_options() 
                    print(f'word.{level} all:{new_last_node.get_sequence()} word:{word} duplicates:{duplicates}')
                   
                    is_new = self.options.append(pos, new_last_node)
                    
                    i = len(word)
                    new_remaining = str(remaining[i:])
                    if new_last_node.get_sum_length_sequence() + len(new_remaining) != len(self.sentence):
                        print(f'word.{level} diff {new_last_node.get_sum_length_sequence()}+{len(new_remaining)}={len(self.sentence)}')
                    
                    new_commands.append(Command(level+1, i, new_last_node, new_remaining))
        
        #print(f'loop. nb_options:{nb_options} new_commands:{len(new_commands)}')
        self.valid_words_map.update(new_sequences)

        return new_commands, nb_options

In [ ]:
def count_runner(target_sentence, words):
    print(f'nb words {len(words)}')
             
    word_encoder = WordEncoder()
    word_encoder.add_word_list(words)
    print(f'encoded words {word_encoder.get_dict()}')
    print(f'nb encoded words {len(word_encoder.get_dict())}')

    context = Context(word_encoder=word_encoder)
    decoder = SentenceDecoder(context)
    res = decoder.decode(target_sentence)

    return res

# -------------------------------------

# unit tests

### empty sentence

In [ ]:
morse_sentence = '' 
words = ['SE', 'T', 'O'] 

res = count_runner(morse_sentence, words)

print(f'count: {res}')

assert res == 0

### empty words

In [ ]:
morse_sentence = '-' # T 
words = [] 

res = count_runner(morse_sentence, words)

print(f'count: {res}')

assert res == 0

### one word

In [ ]:
morse_sentence = '-' # T 
words = ['T'] 

res = count_runner(morse_sentence, words)

print(f'count: {res}')

assert res == 1

In [ ]:
morse_sentence = '--' # T 
words = ['T', 'X', 'M'] 

res = count_runner(morse_sentence, words)

print(f'count: {res}')

assert res == 2

### One letter - one option

In [ ]:
morse_sentence = '-' # T
words = ['SE', 'T', 'O'] 

res = count_runner(morse_sentence, words)
print(f'count: {res}')

assert res == 1 # T

### very few words

In [ ]:
morse_sentence = '-.-.' # TETE
words = ['TE'] 

res = count_runner(morse_sentence, words)
print(f'count: {res}')

assert res == 1 # TE TE

In [ ]:
morse_sentence = '-.-.' # TETE
words = ['T','E'] 

res = count_runner(morse_sentence, words)
print(f'count: {res}')

assert res == 1 # TE TE

### short message - multiple options

In [ ]:
morse_sentence = '....' # E . I .. S ... H ....
words = ['EIE', 'SE', 'ES', 'H', 'L', 'O'] 

res = count_runner(morse_sentence, words)
print(f'count: {res}')

assert res == 4 # EIE, ES, H, SE 

### no match

In [ ]:
morse_sentence = '....' # E . I .. S ... H ....
words = ['S', 'L', 'O'] 

res = count_runner(morse_sentence, words)
print(f'count: {res}')

assert res == 0 # no match

### short message - multiple options with permutations

In [ ]:
morse_sentence = '.....' # confusion EH/HE
words = ['HEL', 'HE', 'EH', 'O'] 

res = count_runner(morse_sentence, words)
print(f'count: {res}')

assert res == 2 # HE, EH

### short message - one option

In [ ]:
morse_sentence = '......-..' # HEL single option
words = ['HEL', 'O'] 

res = count_runner(morse_sentence, words)
print(f'count: {res}')

assert res == 1 # HEL

### short message - multiple options with partial match

In [ ]:
morse_sentence = '......-..' # HEL or HE L
words = ['HEL', 'HE', 'L', 'O'] 

res = count_runner(morse_sentence, words)
print(f'count: {res}')

assert res == 2 # HEL, HE L  -- fix stops when HE L is found and never reach HEL

### short message - multiple options with partial match and permutations

In [ ]:
morse_sentence = '......-..' # HEL with confusion EH/HE
words = ['HEL', 'HE', 'EH', 'L', 'O'] 

res = count_runner(morse_sentence, words)
print(f'count: {res}')

assert res == 3 # HEL, HE L, EH L

### short sample message - multiple options

In [ ]:
morse_sentence = '......-...-..---' # HELLO 
words = ['HELL', 'HELLO', 'WORLD', 'OWORLD', 'TEST', 'L', 'O'] 

res = count_runner(morse_sentence, words)
print(f'count: {res}')

assert res == 2 # HELLO, HELL O

### short sample message - multiple options with permutations

In [ ]:
morse_sentence = '......-...-..---' # HELLO with confusion EH/HE
words = ['HELL', 'HELLO', 'WORLD', 'OWORLD', 'TEST', 'HE', 'EH', 'L', 'O'] 

res = count_runner(morse_sentence, words)
print(f'count: {res}')

assert res == 4 # HELLO, HELL O, HE L L O, EH L L O

### sample message

In [ ]:
morse_sentence = '......-...-..---.-----.-..-..-..' # HELLOWORLD
words = ['HELL', 'HELLO', 'WORLD', 'OWORLD', 'TEST'] 

start = time.perf_counter()
res = count_runner(morse_sentence, words)
stop = time.perf_counter()
print(f"duration {stop-start}", file=sys.stderr, flush=True)

print(f'count: {res}')

assert res == 2 # HELLO WORLD, HELL OWORLD

### other sample

In [ ]:
morse_sentence = '--.-------..' # HELLOWORLD
words = ['GOD', 'GOOD', 'MORNING', 'G', 'HELLO'] 

start = time.perf_counter()
res = count_runner(morse_sentence, words)
stop = time.perf_counter()
print(f"duration {stop-start}", file=sys.stderr, flush=True)

print(f'count: {res}')

assert res == 1 # HELLO WORLD, HELL OWORLD

count avec startwith 
unitaire  5.9784999997702926e-05

# -------------------------------------

# long string

### long string generation fixture

In [ ]:
import random
def generate_random_morse_sentence(length, signs=None, seed=1234, chunk_size=None):
    random.seed(seed)
    sentence = []
    stats = {}
    tokens = []
    current_token = []
    if not chunk_size: chunk_size = random.randint(0, 4) + random.randint(0, 16)

    if not signs: signs = list(alphabet_map.keys())
    max_sign = len(signs) -1
    for s in signs:
        stats[s] = 0
    for i in range(length):
        letter = signs[random.randint(0, max_sign)]
        sentence.append(alphabet_map[letter])
        stats[letter] += 1
        current_token.append(letter)
        if len(current_token) >= chunk_size:
            tokens.append(''.join(current_token))
            current_token = []
    
    if current_token:
        tokens.append(''.join(current_token))
         
    unique_tokens = list(set(tokens))
    
    return ''.join(sentence), stats, unique_tokens

In [ ]:
alphabet_map = morse.get_encoded_letters_map()

generated_morse_sentence, stats, tokens = generate_random_morse_sentence(1, signs='E', chunk_size=5)
assert len(generated_morse_sentence) == 1
assert stats['E'] == 1

generated_morse_sentence, stats, tokens = generate_random_morse_sentence(2, signs='ET', chunk_size=5)
assert len(generated_morse_sentence) == 2
assert stats['E'] == 1
assert stats['T'] == 1

generated_morse_sentence, stats, tokens = generate_random_morse_sentence(2, chunk_size=5)
assert len(generated_morse_sentence) == 7
assert stats['O'] == 1
assert stats['Y'] == 1

In [ ]:
generated_morse_sentence, stats, tokens = generate_random_morse_sentence(10, signs='ET', chunk_size=5)
print(tokens)
assert len(tokens) == 2
assert len(tokens[0]) == 5
assert len(tokens[1]) == 5
assert tokens[0] == 'TEEEE'
assert tokens[1] == 'EETEE'

generated_morse_sentence, stats, tokens = generate_random_morse_sentence(10, signs='E', chunk_size=5)
print(tokens)
assert len(tokens) == 1
assert len(tokens[0]) == 5
assert tokens[0] == 'EEEEE'

generated_morse_sentence, stats, tokens = generate_random_morse_sentence(8, signs='E', chunk_size=5)
print(tokens)
assert len(tokens) == 2
assert len(tokens[0]) == 3
assert len(tokens[1]) == 5
assert tokens[0] == 'EEE'
assert tokens[1] == 'EEEEE'

generated_morse_sentence, stats, tokens = generate_random_morse_sentence(8, chunk_size=2)
print(tokens)
assert len(tokens) == 4
assert len(tokens[0]) == 2
assert tokens[0] == 'YO'

generated_morse_sentence, stats, tokens = generate_random_morse_sentence(40)
print(tokens)
assert len(tokens) == 7
assert tokens[0] == 'ACZSBV'


### long string - 1-char word - 1 option - stackoverflow

assume count for 1 word is 1

In [ ]:
morse_sentence, stats, tokens = generate_random_morse_sentence(4000, signs='E')
print(f'stats {stats}')
print(f'length {len(morse_sentence)}')
words = ['E'] 

start = time.perf_counter() 
res = count_runner(morse_sentence, words)
stop = time.perf_counter()
print(f"duration {stop-start}", file=sys.stderr, flush=True)

print(f'count: {res}')

assert res == 1

### long string - 2 1-char words - multiple options - stackoverflow

In [ ]:
morse_sentence, stats, tokens = generate_random_morse_sentence(4000, signs='ET')
print(f'stats {stats}')
print(f'length {len(morse_sentence)}')
words = ['E', 'T'] 

start = time.perf_counter()
res = count_runner(morse_sentence, words)
stop = time.perf_counter()
print(f"duration {stop-start}", file=sys.stderr, flush=True)

print(f'count: {res}')

assert res == 1

### long string - few words - multiple options

issue = a large number of words ':' -> execeed recursion limit

In [ ]:
morse_sentence, stats, tokens = generate_random_morse_sentence(40, signs='E', chunk_size=5)
print(f'stats {stats}')
print(f'length {len(morse_sentence)}')
words = tokens
print(f'nb words {len(words)}')

start = time.perf_counter()
res = count_runner(morse_sentence, words)
stop = time.perf_counter()
print(f"duration {stop-start}", file=sys.stderr, flush=True)

print(f'count: {res}')

assert res == 1

### long string - few words 

In [ ]:
morse_sentence, stats, tokens = generate_random_morse_sentence(4000, signs='ET')
print(f'stats {stats}')
print(f'length {len(morse_sentence)}')
words = tokens
print(f'nb words {len(words)}')

start = time.perf_counter()
res = count_runner(morse_sentence, words)
stop = time.perf_counter()
print(f"duration {stop-start}", file=sys.stderr, flush=True)

print(f'count: {res}')

assert res == 1

### long string - more words 

In [ ]:
morse_sentence, stats, tokens = generate_random_morse_sentence(4000)
print(f'stats {stats}')
print(f'length {len(morse_sentence)}')
words = tokens
print(f'nb words {len(words)}')

start = time.perf_counter()
res = count_runner(morse_sentence, words)
stop = time.perf_counter()
print(f"duration {stop-start}", file=sys.stderr, flush=True)

print(f'count: {res}')

assert res == 1

### long sentence - permutations

In [ ]:
morse_sentence, stats, tokens = generate_random_morse_sentence(4000, signs='EISH')
print(f'stats {stats}')
print(f'length {len(morse_sentence)}')
words = tokens
print(f'nb words {len(words)}')

start = time.perf_counter()
res = count_runner(morse_sentence, words)
stop = time.perf_counter()
print(f"duration {stop-start}", file=sys.stderr, flush=True)

print(f'count: {res}')

assert res == 1

## long sentence - with repeating pattern

In [ ]:
morse_sentence = '.-.-.-.-.-.-.-.-' # ETETETETETETETET
words = ['E', 'T'] 

start = time.perf_counter()
res = count_runner(morse_sentence, words)
stop = time.perf_counter()
print(f"duration {stop-start}", file=sys.stderr, flush=True)

print(f'count: {res}')

assert res == 1 # HELLO WORLD, HELL OWORLD

# -------------------------------------

# many words

In [ ]:
import random
def generate_words(nb, max_length=20, signs=None, seed=1234):
    random.seed(seed)
    words = []
    
    for i in range(nb):
        current_token = []
        size = random.randint(0, 4) + random.randint(0, max_length)

        #if not signs: signs = list(alphabet_map.keys())
        #max_sign = len(signs) -1
        #for s in signs:
        #    stats[s] = 0
        for i in range(size):
            letter = random.randint(0, 25)
            current_token.append('ABCDEFGHIJKLMNOPQRSTUVWXYZ'[letter])
            #stats[letter] += 1
        words.append(''.join(current_token))
         
    return words

In [ ]:
alphabet_map = morse.get_encoded_letters_map()
nb = 10000

start = time.perf_counter()
words = generate_words(nb, seed=1234)
stop = time.perf_counter()
print(f"duration {stop-start}", file=sys.stderr, flush=True)

assert len(words) == nb
print(words[0])

In [ ]:
start = time.perf_counter()
lengths = {w:len(w) for w in words}
stop = time.perf_counter()
print(f"duration {stop-start}", file=sys.stderr, flush=True)

In [ ]:
start = time.perf_counter()
for w in words:
    n = lengths[w]
stop = time.perf_counter()
print(f"duration {stop-start}", file=sys.stderr, flush=True)